In [1]:
import numpy as np
import sys
import itertools as itertools
sys.path.append("..")
sys.path.append("../..")

import matplotlib.pyplot as plt
from python.Hubbard import Hubbard_model_with_filling, Double_occupancy, get_filling
from python.DMRG import DMRG
from python.Zippers import MPO_to_Hamiltonian, contract_MPS
from python.Canonical_Form import get_Neumann_entropy

In [56]:
n_sites = 5

hopping_t = 1.0
interaction_U = 2.0
chemical_potential = 1.0

Hamiltonian = Hubbard_model_with_filling(
    n_sites=n_sites, hopping_t=hopping_t,
    interaction_U=interaction_U, chemical_potential=chemical_potential,
)

MPO_transposed = [ham.transpose(2, 3, 0, 1) for ham in Hamiltonian]

In [ ]:
"""
Get exact matrix (exponential barrier)
"""

matrix = MPO_to_Hamiltonian(MPO_transposed)
eigvals, _ = np.linalg.eigh(matrix)
eigvals[:10]

array([-8.55415139, -8.55415139, -8.16604011, -8.16604011, -7.77446918,
       -7.77446918, -7.52435237, -7.52435237, -7.52435237, -7.52435237])

In [58]:
np.linalg.norm(matrix - matrix.conj().T)

0.0

In [ ]:
NKeep = 20
NSweep = 10
Krylov_bases = 5 #* 5 is usually enough
Lanczos_cutoff = 1e-2
iterative_diag = True
two_site = True
verbose = True

In [60]:
ground_energies, ground_times, ground_MPS = DMRG(
    Hamiltonian = Hamiltonian,
    NKeep = NKeep,
    NSweep = NSweep,
    Krylov_bases = Krylov_bases,
    Lanczos_cutoff = Lanczos_cutoff,
    iterative_diag = iterative_diag,
    two_site = two_site,
    verbose = verbose,
)

eigvals[0]

L=5 | NKeep=20 | NSweep=10 | iterative=True | two_site=True | Krylov_bases=5 | Lanczos_cutoff=0.01
iter=0 | energy=-8.3591856 | time=0.0s
iter=1 | energy=-8.5541463 | time=0.0264s
iter=2 | energy=-8.5541463 | time=0.0394s
iter=3 | energy=-8.5541463 | time=0.0522s
iter=4 | energy=-8.5541463 | time=0.0656s
iter=5 | energy=-8.5541463 | time=0.0789s
iter=6 | energy=-8.5541463 | time=0.0903s
iter=7 | energy=-8.5541463 | time=0.1s
iter=8 | energy=-8.5541463 | time=0.11s
iter=9 | energy=-8.5541463 | time=0.12s
iter=10 | energy=-8.5541463 | time=0.13s


-8.5541513932622

In [61]:
excited_energies, excited_times, excited_MPS = DMRG(
    Hamiltonian = Hamiltonian,
    NKeep = NKeep,
    NSweep = NSweep,
    Krylov_bases = Krylov_bases,
    Lanczos_cutoff = Lanczos_cutoff,
    iterative_diag = False,
    orthogonal_to_list_of_MPS=[ground_MPS],
    two_site = two_site,
    verbose = verbose,
)

eigvals[1]

L=5 | NKeep=20 | NSweep=10 | iterative=False | two_site=True | Krylov_bases=5 | Lanczos_cutoff=0.01
iter=0 | energy=-2.3729959 | time=0.0s
iter=1 | energy=-8.554003 | time=0.123s
iter=2 | energy=-8.5541364 | time=0.153s
iter=3 | energy=-8.5541364 | time=0.177s
iter=4 | energy=-8.5541364 | time=0.204s
iter=5 | energy=-8.5541364 | time=0.27s
iter=6 | energy=-8.5541364 | time=0.299s
iter=7 | energy=-8.5541364 | time=0.32s
iter=8 | energy=-8.5541364 | time=0.361s
iter=9 | energy=-8.5541364 | time=0.383s
iter=10 | energy=-8.5541364 | time=0.442s


-8.554151393262199

In [62]:
get_Neumann_entropy(ground_MPS)

array([1.6311999 , 1.34413932, 1.34677498, 1.62915346])

In [63]:
get_Neumann_entropy(excited_MPS)

array([1.63077494, 1.34599653, 1.34515945, 1.63128744])

In [64]:
get_filling(ground_MPS).sum()

4.999999998830106

In [65]:
get_filling(excited_MPS).sum()

(4.999976575593868-2.3592239273284576e-16j)